In [11]:
using MathOptInterface;
const MOI = MathOptInterface;
const MOIU = MOI.Utilities;

$$
\begin{array}{rl}
    \max & \mathbf{c}^{\small\mathbf{\text{T}}}\mathbf{x} \\
    \text{s.t.} & \mathbf{w}^{\small\mathbf{\text{T}}}\mathbf{x} \le C\\
    ~ & \mathbf{x}_i \in \{0, 1\}
\end{array}
$$

In [13]:
# Model definition
# References:
# [1] https://jump.dev/MathOptInterface.jl/stable/tutorials/example/

model = MOIU.Model{Float64}()

n = 3;
c = [1.0, 2.0, 3.0];
w = [0.3, 0.5, 1.0];
C = 3.2;

x = MOI.add_variables(model, n);

# ---------
# Objective
# ---------
MOI.set(model, MOI.ObjectiveSense(), MOI.MAX_SENSE)

MOI.set(
   model,
   MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}(),
   MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(c, x), 0.0),
);

# -----------
# Constraints
# -----------
MOI.add_constraint(
   model,
   MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(w, x), 0.0),
   MOI.LessThan(C),
);

for xᵢ in x
   MOI.add_constraint(model, xᵢ, MOI.ZeroOne())
end

print(model);

In [32]:
module ToQUBO
    # -*- ToQUBO.jl -*-
    using JuMP, MathOptInterface
    const MOI = MathOptInterface
    const MOIU = MathOptInterface.Utilities
    const MOIB = MathOptInterface.Bridges

    const SVF = MOI.SingleVariable
    const VVF = MOI.VectorOfVariables
    const SAF{T} = MOI.ScalarAffineFunction{T}
    const VAF{T} = MOI.VectorAffineFunction{T}
    const SQF{T} = MOI.ScalarQuadraticFunction{T}
    const VQF{T} = MOI.VectorQuadraticFunction{T}

    const VI = MOI.VariableIndex
    const CI = MOI.ConstraintIndex

    # -*- structures.jl -*-
    """
    Storing the problem as pairs {vars => constant} looks like a good a idea
    """
    struct Posiform{T <: Real}
        pairs::Dict{Tuple{VI}, T}
        degree::Int
    
        function Posiform(d::Dict{Tuple{VI}, T}) where {T}
            return new{T}(d, 0)
        end
    end

    mutable struct QUBOModel{T}
        invert_sense::Bool
    
        function QUBOModel{T}() where {T}
            return new{T}(false)
        end
    end

    # -*- supported.jl -*-

    # Objective Support
    function supported_objective(model::MOI.ModelLike)
        F = MOI.get(model, MOI.ObjectiveFunctionType())
        if !__supported_objective(F)
            error("Objective functions of type ", F, " are not implemented")
        end
        return
    end

    __supported_objective(::Type) = false

    # Constraint Support
    function supported_constraints(model::MOI.ModelLike)
        for (F, S) in MOI.get(model, MOI.ListOfConstraints())
            if !__supported_constraint(F, S)
                error(
                    "Constraints of function ",
                    F,
                    " in the Set ",
                    S,
                    " are not implemented",
                )
            end
        end
        return
    end

    __supported_constraint(::Type, ::Type) = false

    
    # toqubo
    """
    """
    function toqubo(model::MOI.ModelLike)
    
        # Support Validation
        supported_objective(model)
        supported_constraints(model)
    
        # Objective Sense
        
    end;
end;

#ToQUBO.toqubo(model)

In [74]:
toqubo(model)

var: MathOptInterface.VariableIndex(1)
var: MathOptInterface.VariableIndex(2)
var: MathOptInterface.VariableIndex(3)
(MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64})


1-element Vector{Nothing}:
 nothing